<a href="https://colab.research.google.com/github/riyaaaarane/yield-lstm/blob/main/models_yield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from xgboost import XGBRegressor



In [64]:
df = pd.read_csv('/content/four-features-all-villages - all_data(5 villages) - Copy of mahalingpur_farm_data_dailyagrre (1).csv.csv')

In [65]:
df.head()

,date,precipitation_mm,temperature_C,lai,fapar,yield,village
0,2007-01-01,0.0,30.035635,11.029443,54.285325,108.25,mahalingpur
1,2007-01-02,0.0,29.799342,11.143477,54.562126,108.25,mahalingpur
2,2007-01-03,0.0,29.572417,11.257510,54.838927,108.25,mahalingpur
3,2007-01-04,0.0,30.142984,11.371544,55.115728,108.25,mahalingpur
4,2007-01-05,0.0,29.417248,11.485578,55.392530,108.25,mahalingpur


In [66]:
df.shape

(10960, 7)

In [67]:
df['yield_lag1'] = df.groupby('village')['yield'].shift(1)
df['yield_lag2'] = df.groupby('village')['yield'].shift(2)

In [68]:
df

,date,precipitation_mm,temperature_C,lai,fapar,yield,village,yield_lag1,yield_lag2
0,2007-01-01,0.0,30.035635,11.029443,54.285325,108.25,mahalingpur,NaN,NaN
1,2007-01-02,0.0,29.799342,11.143477,54.562126,108.25,mahalingpur,108.25,NaN
2,2007-01-03,0.0,29.572417,11.257510,54.838927,108.25,mahalingpur,108.25,108.25
3,2007-01-04,0.0,30.142984,11.371544,55.115728,108.25,mahalingpur,108.25,108.25
4,2007-01-05,0.0,29.417248,11.485578,55.392530,108.25,mahalingpur,108.25,108.25
...,...,...,...,...,...,...,...,...,...
10955,2012-12-27,0.0,30.148230,7.581671,44.506220,1839.56,munyal,1839.56,1839.56
10956,2012-12-28,0.0,30.560281,7.836705,44.111776,1839.56,munyal,1839.56,1839.56
10957,2012-12-29,0.0,32.335009,7.326638,43.717331,1839.56,munyal,1839.56,1839.56
10958,2012-12-30,0.0,31.811971,7.581671,44.111776,1839.56,munyal,1839.56,1839.56


In [69]:
# Drop rows with any NaN values
df_cleaned = df.dropna()

X = df_cleaned.drop(['date', 'yield', 'village'], axis=1)
y = df_cleaned['yield']

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [71]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [72]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

LinearRegression()

In [73]:
model_lr.score(X_test, y_test)

0.9939509044688477

In [74]:
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [75]:
model_rf.score(X_test, y_test)

0.8470093554087005

In [76]:
model_knn = KNeighborsRegressor(n_neighbors=5)
model_knn.fit(X_train, y_train)

KNeighborsRegressor()

In [77]:
model_knn.score(X_test, y_test)

0.17200945394276457

In [78]:
model_xgb = XGBRegressor(
    n_estimators=300,        # number of trees
    learning_rate=0.05,      # step size shrinkage
    max_depth=6,             # depth of each tree
    subsample=0.8,           # sample fraction
    colsample_bytree=0.8,    # feature fraction
    reg_alpha=0.1,           # L1 regularization
    reg_lambda=1.0,          # L2 regularization
    random_state=42,
    tree_method="hist"       # efficient for large data
)

model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=300,
             n_jobs=None, num_parallel_tree=None, ...)

In [79]:
model_xgb.score(X_test, y_test)

0.49279591176179616

In [83]:
# Ridge Regression
model_lr_ridge = Ridge(alpha=1.0)  # You can adjust alpha
model_lr_ridge.fit(X_train, y_train)
print("Ridge Regression R-squared:", model_lr_ridge.score(X_test, y_test))

Ridge Regression R-squared: 0.9932933555184666


In [84]:
# Lasso Regression
model_lr_lasso = Lasso(alpha=0.1)  # You can adjust alpha
model_lr_lasso.fit(X_train, y_train)
print("Lasso Regression R-squared:", model_lr_lasso.score(X_test, y_test))

Lasso Regression R-squared: 0.9939513562899496


In [85]:
# Elastic Net Regression
model_lr_elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)  # Adjust alpha and l1_ratio
model_lr_elastic_net.fit(X_train, y_train)
print("Elastic Net Regression R-squared:", model_lr_elastic_net.score(X_test, y_test))

Elastic Net Regression R-squared: 0.984249804551523
